In [1]:
# TODO - naming, state, key...
from contextlib import contextmanager


"""
State Management.

Everything that needs to store some data delegates it to a State object, stored at _state.

Centralising data seperates data storage from the API presented to users.
This opens up potential to implement different user facing APIs e.g. drawBot on top of the
same underlying state.

State Management tracks the current fill, stroke, transformation etc.

Program->Nodebox API->[State[->Cairo Renderer->Output

Program->
"""

class StateException(Exception):
    pass


class State:
    def __init__(self, store, key, data, *args, **kwargs):
        self.key = key
        self.data = data
        self.store = store
        self.store.update(key, data)
        
    def __getitem__(self, key):
        """
        :return:  Values stored in current state.
        >>> state["fill"]
        {"value": (0.1, 0.1, 0.1, 0.1), "mode": "rgba"}
        """
        return self.state[key]
    
    def __repr__(self):
        return f"<{self.__class__.__name__} {self.state}>"


class ContextManagerState(State):
    """
    State that may optionally be used as in a with statement.
    
    When used as ContextManager new state is pushed onto a stack,
    and popped when the Context Manager exits.
    
    Users may want to raise an exception under some circumstances,
    e.g. many APIs do not allow the color to change when during a Path.
    
    ContextManagerState can raise an arbitrary exception by setting
    cm_raises_exception to a tuple of (exception, args, kwargs),
    args and kwargs are optional.
    
    :param cm_raises_exception: None if use as a context manager is allowed.
    :param cm_raises_exception: (exception, args, kwargs) 
    exception to raise on use as context manager.
    """
    def __init__(self, store, key, data, 
                 cm_raises_exception=None, 
                 *args, **kwargs):
        self.cm_raises_exception = cm_raises_exception
        self.entered = False

        if cm_raises_exception:
            self.restore_state = None
        else:
            # restore state for when used as context manager.
            self.restore_state = {**store.get(key)}
        
        State.__init__(self, store, key, data, **kwargs)
    
    def __enter__(self):
        """
        Enter context manager.
        Since state was already set in the constructor it is nessacary
        to reset the state to that saved in self.restore_state, then
        push the desired state stored in self.state to the stack.
        
        Throw exception if self.cm_raises_exception is set.
        """
        if self.cm_raises_exception:
            ex = self.cm_raises_exception[0]            
            if len(self.cm_raises_exception) == 1:
                raise ex()
                
            args = self.cm_raises_exception[1]
            if len(self.cm_raises_exception) == 2:
                raise ex(*args)
                
            kwargs = self.cm_raises_exception[2]
            raise ex(*args, **kwargs)
            
        # Avoid expensive list insert: restore state, then pushing desired state. 
        self.store.update(self.key, self.restore_state)
        self.store.push(self.key, self.state)
        
    def __exit__(self, *args):
        self.store.pop()
        self.state = self.restore_state


class ColorState(ContextManagerState):
    def __init__(self, store, key, **kwargs):
        """
        :param store: Backend to store color data.
        :param mode: rgba| indexed
        :param value: channel data
        """
        ContextManagerState.__init__(self, store, key,
                                     {"mode": kwargs.pop("mode"),
                                      "value": kwargs.pop("value")},
                                     **kwargs)
        
class Color:
    def __init__(self, ctx, *args, **kwargs):
        self._state = {"mode": kwargs.pop("mode", "rgba"), "value": args}
#         self._state = ColorState(ctx.state, 
#                                  ctx.state.current_key,
#                                  value=args,
#                                  mode=kwargs.pop("mode", "rgba"),
#                                  cm_raises_exception=kwargs.pop("cm_raises_exception", False))
        self.ctx = ctx
        
    @staticmethod
    def from_state(ctx, state, **kwargs):
        _kwargs = {**state}
        _kwargs.update(kwargs)
        value = _kwargs.pop("value")
        return Color(ctx, *value, **_kwargs)
        
    @property
    def r(self):
        return self._state["r"]
    
    @property
    def g(self):
        return self._state["g"]

    @property
    def b(self):
        return self._state["b"]

    @property
    def a(self):
        return self._state["a"]
    
    @property
    def mode(self):
        return self._state["mode"]
    
    def __enter__(self):
        return ColorState(self.ctx.state_manager, 
                          self._state.key,
                          mode=self._state["mode"],
                          value=self._state["value"],
                          cm_raises_exception=self._state.cm_raises_exception)
    
    def __exit__(self, *args):
        return False
    
    def __repr__(self):
        return f"<Color {self._state['value']} mode={self.mode}>"


        
        
class BlendModeState(ContextManagerState):
    def __init__(self, store, key, state, blendmode, **kwargs):
        """
        :param store: Backend to store color data.
        :param blendmode: rgba, indexed
        """
        ContextManagerState.__init__(self, store, key,
                                     {"blendmode": blendmode})


class PathState(State):
    def __init__(self, store, fill, stroke):
        State.__init__(self, store, "path",
                       {
                           "fill": fill, 
                           "stroke": stroke,
                           "elements": []
                       },
                       cm_raises_exception)


class StateManager:
    """
    Data store for state related to handling drawing.
    State is handled seperately to the API users see,
    this allows
    
    Current State is a dict of dicts, with entries like
    
    {"fill": {"mode": "rgba", "value", (1, 1, 1, 1)}}
    
    These are held in a list to implement a stack,
    
    If new state needs to be pushed, then current state is shallow
    copied to a new dict, containing another new dict for the
    key to be updated.
    """
    def __init__(self, states):
        self.stack = []
        self.stack.append(states)
        self.cm_key = []
        
    def update(self, key, state):
        """
        Update state for given key.
        
        :param key: location to save data.
        :param state:  dict of data to save under this key.
        """
        self.get(key).update(state)
    
    def get(self, key):
        return self.stack[-1][key]
    
    @property
    def states(self):
        return self.stack[-1]
    
    def pop(self):
        del self.stack[-1]

    def push(self, key, state):
        states = {**self.states, **{key: {**state}}}
        self.stack.append(states)
        return len(self.stack) -1
    
    @contextmanager
    def key(self, key):
        """
        Context manager that allows the setting of a current key to
        save data under.
        
        The key can then be fetched with the current_key property.
        
        >>> sm = StateManager()
        >>> with sm.key("fill"):
        ...     print(sm.current_key)
        "fill"
        """
        self.cm_key.append(key)
        try:
            yield self.get(key)
        finally:
            del self.cm_key[-1]

    @property
    def current_key(self):
        """
        Get key that was set using the key(...) context manager.
        """
        try:
            return self.cm_key[-1]
        except IndexError:
            raise StateException("No current key set, .current_key can only be called when using the key(...) context manager.")
    
    def __repr__(self):
        return f"<StateStack {self.stack}>"
        

class Context:
    def __init__(self):
        initial_state = {
            "fill": {"mode": "rgba", "value": (1., 1., 1., 1.)},
            "stroke": {"mode": "rgba", "value": (0., 0., 0., 1.)},
        }
        self.state = StateManager(initial_state)

    def fill(self, *rgba):
        if not rgba:
            return Color.from_state(self, self.state.get("fill"))
        color = Color(self, *rgba)
        self.state.update("fill", color._state)
        return color

        #     def fill(self, *rgba):
#         with self.store.key("fill") as state:
#             if not rgba:
#                 return Color.from_state(self, state)
#             return Color(self, *rgba)
    
    def __repr__(self):
        return f"<Context {self.state_manager}>"
    
ctx = Context()
#print(ctx.store.stack[-1])
#color = ctx.fill(1., 0., 1., 1.)
print(ctx.state.states)
print()
print(ctx.fill())
#print(ctx)
with ctx.fill(0.0, 0, 0, 1.0):
    print("Inside CM")
    print(ctx.state.states)
    
print("Outside CM")
print(ctx.fill())
print(ctx.store.states)

{'fill': {'mode': 'rgba', 'value': (1.0, 1.0, 1.0, 1.0)}, 'stroke': {'mode': 'rgba', 'value': (0.0, 0.0, 0.0, 1.0)}}

<Color (1.0, 1.0, 1.0, 1.0) mode=rgba>


AttributeError: 'Context' object has no attribute 'state_manager'